# Instructions

For this assignment you will use PyTorch instead of EDF to implement and train neural networks. The experiments in this assignment will take a long time to run without a GPU, but you can run the notebook remotely on Google Colab and have access to GPUs for free -- in this case you don't have to worry about installing PyTorch as it is available by default in Google Colab's environment.

In case you will be running the experiments in your own machine, you should install PyTorch -- there are multiple tutorials online and it is especially easy if you're using Anaconda. Check https://pytorch.org/tutorials/ for some PyTorch tutorials -- this assignment assumes that you know the basics like defining models with multiple modules and coding up functions to train models with PyTorch optimizers. To

To use Google Colab, you should access https://colab.research.google.com/ and upload this notebook to your workspace. To use a GPU, go to Edit -> Notebook settings and select GPU as the accelerator.

Unlike previous assignments, in this one you will have to do some writing instead of just coding. Try to keep your answers short and precise, and you are encouraged to write equations if needed (you can do that using markdown cells). You can also use code as part of your answers (like plotting and printing, etc). Blue text indicates questions or things that you should discuss/comment, and there will red "ANSWER (BEGIN)" and "ANSWER (END)" markdown cells to indicate that you should add cells with your writeup between these two. **Make sure not to redefine variables or functions in your writeup, which can change the behavior of the next cells.**

Finally, you might have to do minor changes to the provided code due to differences in python/pytorch versions. You can post on piazza if there's a major, non-trivial change that you had to do (so other students can be aware of it and how to proceed), but for minor changes you should just apply them and keep working on the assignment.

In [12]:
import torch, math, copy
import numpy as np
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

# From Shallow to Deep Neural Networks

The main goal of this assignment is to develop a better understanding of how the depth of a network interacts with its trainability and performance.

In the previous assignment you likely observed difficulties in training sigmoid and ReLU networks with over ~8 layers, which is typically associated with 'vanishing' or 'exploding' gradients. As you will see, some of the biggest achievements in deep learning have been the development of techniques that enable deeper networks to be successfully trained, and without them deep networks are notoriously difficult to train successfully.

You will be working with the MNIST dataset, which will be downloaded and loaded in the cell below.

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

train_dataset = datasets.MNIST("data", train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)

test_dataset = datasets.MNIST("data", train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 110799424.48it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 41300952.55it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 23596235.31it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 23007884.99it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



Fill the missing code below. In both train_epoch and test, total_correct should be the total number of correctly classified samples, while total_samples should be the total number of samples that have been iterated over.

In [48]:
def train(epochs, model, criterion, optimizer, train_loader, test_loader):
    for epoch in range(epochs):
        train_err = train_epoch(model, criterion, optimizer, train_loader)
        test_err = test(model, test_loader)
        print('Epoch {:03d}/{:03d}, Train Error {:.2f}% || Test Error {:.2f}%'.format(epoch+1, epochs, train_err*100, test_err*100))
    return train_err, test_err

def train_epoch(model, criterion, optimizer, loader):
    total_correct = 0.
    total_samples = 0.

    for batch_idx, (data, target) in enumerate(loader):
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()

        # insert code to feed the data to the model and collect its output
        output = model(data)

        # insert code to compute the loss from output and the true target
        loss = criterion(output, target)

        # insert code to update total_correct and total_samples
        # total_correct: total number of correctly classified samples
        # total_samples: total number of samples seen so far
        total_correct += torch.sum(torch.argmax(output, 1) == target).item()
        total_samples += target.shape[0]

        # insert code to update the parameters using optimizer
        # be careful in this part as an incorrect implementation will affect
        # all your experiments and have a significant impact on your grade!
        # in particular, note that pytorch does --not-- automatically
        # clear the parameter's gradients: check tutorials to see
        # how this can be done with a single method call.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return 1 - total_correct/total_samples

def test(model, loader):
    total_correct = 0.
    total_samples = 0.
    model.eval()

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(loader):
            if torch.cuda.is_available():
                data, target = data.cuda(), target.cuda()

            # insert code to feed the data to the model and collect its output
            output = model(data)

            # insert code to update total_correct and total_samples
            # total_correct: total number of correctly classified samples
            # total_samples: total number of samples seen so far
            total_correct += torch.sum(torch.argmax(output, 1) == target).item()
            total_samples += target.shape[0]

    return 1 - total_correct/total_samples

### CNN with Tanh activations

Next, you should implement a baseline model so you can check how increasing the number of layers can make a network considerably harder to train, given that no additional methods such as residual connections and normalization layers are adopted.

Finish the implementation of CNNtanh below, carefully following the specifications:

The model should have exactly 'k' many convolutional layers, followed by a linear (fully-connected) layer that actually outputs the logits for each of the 10 MNIST classes.

The network should consist of 3 stages, each with k/3 many convolutional layers (you can assume k is divisible by 3). Each conv layer should have a 3x3 kernel, a stride of 1 and a padding of 1 pixel (such that the output of the convolution has the same height and width as its input).

It should also have an average pooling layer at the end of each stage, with a 2x2 window (hence halving the spatial dimensions), and the number of channels should double from one stage to the other (starting with 4 in the first stage). Moreover, a Tanh activation should follow each convolution layer.

When k=3, for example, the network should be:

1. Stage 1 (1x28x28 input, 4x14x14 output):
    1. Conv layer with 1 input channel and 4 output channels, 3x3 kernel, stride=padding=1
    2. Tanh activation
    3. Average Pool with 2x2 kernel and stride 2
2. Stage 2 (4x14x14 input, 8x7x7 output):
    1. Conv layer with 4 input channels and 8 output channels, 3x3 kernel, stride=padding=1
    2. Tanh activation
    3. Average Pool with 2x2 kernel and stride 2
3. Stage 3 (8x7x7 input, 16x3x3 output):
    1. Conv layer with 8 input channels and 16 output channels, 3x3 kernel, stride=padding=1
    2. Tanh activation
    3. Average Pool with 2x2 kernel and stride 2
4. Fully-connected layer with 16 * 3 * 3=144 input dimension and 10 output dimension

Note that the model should not have any activation after the fully-connected layer: the PyTorch loss module that will be adopted takes logits as input and not class probabilities.

In contrast to the network exemplified above with k=3, when k=6 it should have two conv layers per stage instead of one (each one with a tanh activation following it).

Lastly, do not change the code block with a for loop in the end of init: its purpose to randomly initialize the parameters of the conv layers by sampling from a Gaussian with zero mean and 0.05 deviation.

In [ ]:
class CNNtanh(nn.Module):
    def __init__(self, k):
        super(CNNtanh, self).__init__()

        # write code here to instantiate layers
        # for example, self.conv = nn.Conv2d(1, 4, 3, 1, 1)
        # creates a conv layer with 1 input channel, 4 output
        # channels, a 3x3 kernel, and stride=padding=1
        self.layer_stack = nn.Sequential()
        inp = 1

        for stage in range(3): ## ANS: Loop over stages
          inp = 1 if stage == 0 else int(math.pow(2,stage+1))
          ## ANS: set input size as power of 2 except as 1 for first stage
          ## i.e. 1,4,8
          out = int(math.pow(2,stage+2))
          ## ANS: set output size as power of 2
          ## i.e. 4,8,16
          for i in range(k//3):
            self.layer_stack.add_module(f"stage{stage+1}conv{i+1}", nn.Conv2d(inp,out,3,1,1))
            self.layer_stack.add_module(f"stage{stage+1}tanh{i+1}", nn.Tanh())
            inp = out ## ANS: to move from first layer first stage being 1 -> 4 to second layer first step being 4 -> 4
          self.layer_stack.add_module(f"stage{stage+1}avgpool", nn.AvgPool2d(2,2))

        self.layer_stack.add_module("flatten", nn.Flatten()) ## ANS: flatten before fully connected layer
        self.layer_stack.add_module("fullyconnected", nn.Linear(144,10))

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data.normal_(0, 0.05)
                m.bias.data.zero_()

    def forward(self, input):

        # write code here to define how the output u is computed
        # from the input and the model's layers
        # for example, u = self.conv(input) defines u
        # to be simply the output of self.conv given 'input'
        u = self.layer_stack(input)

        return u

The line below just instantiates the PyTorch Cross Entropy loss, whose inputs should be logits: hence the reason that the CNN should not have an activation after last (feedforward) layer.

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

Now, you should train CNNtanh with different values for k: your goal is to find the largest value for k such that the network achieves less than 20% error (either train or test) in 3 epochs. You should also choose an appropriate learning rate (but do not change the optimizer or the momentum settings!).

Note that CNNs can easily achieve under 2% test error on MNIST, but we're choosing 20% as a threshold since you will be training each network for only 3 epochs.

Remember to use values for k that are divisible by 3. When submitted, your notebook should have the training log of a network with two consecutive values for k (for example, 6 and 9) such that the network is 'trainable' with the smaller one but not 'trainable' with the larger one. It is fine for the training log to include runs with more than two values of k.

In [ ]:
k_list = [6, 9]
lr_list = [0.1]

cnntanh_k_train_test_errs = np.empty((len(k_list), 3))

for lr in lr_list:
  print("Learning rate", lr)
  for idx, k in enumerate(k_list):
    print("\nTraining Tanh CNN with {} layers".format(k))
    model = CNNtanh(k).cuda()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    train_errs, test_errs = train(3, model, criterion, optimizer, train_loader, test_loader)
    cnntanh_k_train_test_errs[idx] = [k, train_errs, test_errs]

Learning rate 0.1

Training Tanh CNN with 6 layers
Epoch 001/003, Train Error 41.72% || Test Error 6.81%
Epoch 002/003, Train Error 4.72% || Test Error 3.24%
Epoch 003/003, Train Error 2.94% || Test Error 2.55%

Training Tanh CNN with 9 layers
Epoch 001/003, Train Error 89.16% || Test Error 88.65%
Epoch 002/003, Train Error 88.90% || Test Error 88.65%
Epoch 003/003, Train Error 88.91% || Test Error 88.65%


### Better Initialization

Next, we will change the initialization of the conv layers and see how it affects the trainability of deep networks. Instead of sampling from a Gaussian with a deviation of 0.05, you should sample from a Gaussian with a deviation $\sigma = \sqrt{\frac{1}{k^2 \cdot C_{out}}}$ or $\sigma = \sqrt{\frac{1}{k^2 \cdot C_{in}}}$, where $k$ is the kernel size ($k=3$ for 3x3 convolutions), $C_{in}$ is the number of input channels, and $C_{out}$ the number of output channels.

The model below should be exactly like CNNtanh except for the standard deviation of the normal distribution used to initialize the conv layers.

The paper 'Understanding the difficulty of training deep feedforward neural networks' by Glorot and Bengio provides some intuition behind such a choice for $\sigma$.

In [ ]:
class CNNtanh_newinit(nn.Module):
    def __init__(self, k):
        super(CNNtanh_newinit, self).__init__()

        # write code here to instantiate layers
        # for example, self.conv = nn.Conv2d(1, 4, 3, 1, 1)
        # creates a conv layer with 1 input channel, 4 output
        # channels, a 3x3 kernel, and stride=padding=1
        self.layer_stack = nn.Sequential()
        inp = 1

        for stage in range(3): ## ANS: Loop over stages
          inp = 1 if stage == 0 else int(math.pow(2,stage+1))
          ## ANS: set input size as power of 2 except as 1 for first stage
          ## i.e. 1,4,8
          out = int(math.pow(2,stage+2))
          ## ANS: set output size as power of 2
          ## i.e. 4,8,16
          for i in range(k//3):
            self.layer_stack.add_module(f"stage{stage+1}conv{i+1}", nn.Conv2d(inp,out,3,1,1))
            self.layer_stack.add_module(f"stage{stage+1}tanh{i+1}", nn.Tanh())
            inp = out ## ANS: to move from first layer first stage being 1 -> 4 to second layer first step being 4 -> 4
          self.layer_stack.add_module(f"stage{stage+1}avgpool", nn.AvgPool2d(2,2))

        self.layer_stack.add_module("flatten", nn.Flatten()) ## ANS: flatten before fully connected layer
        self.layer_stack.add_module("fullyconnected", nn.Linear(144,10))

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # insert code to compute sigma
                sigma =  1/(math.sqrt(m.out_channels)*3) ## choosing \sigma = sqrt(1/(C_in*(k^2)))
                m.weight.data.normal_(0, sigma)
                m.bias.data.zero_()

    def forward(self, input):

        # write code here to define how the output u is computed
        # from the input and the model's layers
        # for example, u = self.conv(input) defines u
        # to be simply the output of self.conv given 'input'
        u = self.layer_stack(input)

        return u

Repeat the procedure of finding the maximum number of layers such that the network is still trainable, this time with CNNtanhinit.

In [ ]:
k_list = [39, 42]
lr_list = [0.1]

cnntanhinit_k_train_test_errs = np.empty((len(k_list), 3))

for lr in lr_list:
  print("Learning rate", lr)
  for idx, k in enumerate(k_list):
    print("\nTraining Tanh CNN + new init with {} layers".format(k))
    model = CNNtanh_newinit(k).cuda()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    train_errs, test_errs = train(3, model, criterion, optimizer, train_loader, test_loader)
    cnntanhinit_k_train_test_errs[idx] = [k, train_errs, test_errs]

Learning rate 0.1

Training Tanh CNN + new init with 39 layers
Epoch 001/003, Train Error 30.38% || Test Error 5.33%
Epoch 002/003, Train Error 4.76% || Test Error 3.13%
Epoch 003/003, Train Error 3.77% || Test Error 2.56%

Training Tanh CNN + new init with 42 layers
Epoch 001/003, Train Error 89.20% || Test Error 89.72%
Epoch 002/003, Train Error 89.00% || Test Error 88.65%
Epoch 003/003, Train Error 89.03% || Test Error 90.20%


### CNN with ELU activations

In this section you should replace the Tanh activations of the previous network for Exponential Linear Units (ELUs). Complete CNNelu below, which should be exactly like CNNtanhinit except for ELU activations instead of Tanh (ELUs are readily available in PyTorch, check its documentation for more details).

In [ ]:
class CNNelu(nn.Module):
    def __init__(self, k):
        super(CNNelu, self).__init__()

        # write code here to instantiate layers
        # for example, self.conv = nn.Conv2d(1, 4, 3, 1, 1)
        # creates a conv layer with 1 input channel, 4 output
        # channels, a 3x3 kernel, and stride=padding=1
        self.layer_stack = nn.Sequential()
        inp = 1

        for stage in range(3): ## ANS: Loop over stages
          inp = 1 if stage == 0 else int(math.pow(2,stage+1))
          ## ANS: set input size as power of 2 except as 1 for first stage
          ## i.e. 1,4,8
          out = int(math.pow(2,stage+2))
          ## ANS: set output size as power of 2
          ## i.e. 4,8,16
          for i in range(k//3):
            self.layer_stack.add_module(f"stage{stage+1}conv{i+1}", nn.Conv2d(inp,out,3,1,1))
            self.layer_stack.add_module(f"stage{stage+1}elu{i+1}", nn.ELU())
            inp = out ## ANS: to move from first layer first stage being 1 -> 4 to second layer first step being 4 -> 4
          self.layer_stack.add_module(f"stage{stage+1}avgpool", nn.AvgPool2d(2,2))

        self.layer_stack.add_module("flatten", nn.Flatten()) ## ANS: flatten before fully connected layer
        self.layer_stack.add_module("fullyconnected", nn.Linear(144,10))

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # insert code to compute sigma
                sigma = 1/(math.sqrt(m.out_channels)*3) ## choosing \sigma = sqrt(1/(C_in*(k^2)))
                m.weight.data.normal_(0, sigma)
                m.bias.data.zero_()

    def forward(self, input):

        # write code here to define how the output u is computed
        # from the input and the model's layers
        # for example, u = self.conv(input) defines u
        # to be simply the output of self.conv given 'input'
        u = self.layer_stack(input)

        return u

Repeat the procedure of finding the maximum number of layers such that the network is still trainable, this time with CNNelu.

In [ ]:
k_list = [84, 87]
lr_list = [0.01]

cnnelu_k_train_test_errs = np.empty((len(k_list), 3))

for lr in lr_list:
  print("Learning rate", lr)
  for idx, k in enumerate(k_list):
    print("\nTraining ELU CNN, with {} layers".format(k))
    model = CNNelu(k).cuda()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    train_errs, test_errs = train(3, model, criterion, optimizer, train_loader, test_loader)
    cnnelu_k_train_test_errs[idx] = [k, train_errs, test_errs]

Learning rate 0.01

Training ELU CNN, with 84 layers
Epoch 001/003, Train Error 82.44% || Test Error 62.46%
Epoch 002/003, Train Error 47.15% || Test Error 26.67%
Epoch 003/003, Train Error 19.69% || Test Error 8.13%

Training ELU CNN, with 87 layers
Epoch 001/003, Train Error 88.88% || Test Error 88.65%
Epoch 002/003, Train Error 88.76% || Test Error 88.65%
Epoch 003/003, Train Error 88.76% || Test Error 88.65%


### CNN with Batch Normalization

Next, you will check how batch normalization can make deep networks easier to train. Implement the network below, which should be exactly like CNNelu except for additional BatchNorm2d layers after each convolution (before the ELU activation).

Note that BatchNorm2d modules require the number of channels as argument -- see the PyTorch documentation for more details.

In [23]:
class CNNeluBN(nn.Module):
    def __init__(self, k):
        super(CNNeluBN, self).__init__()

        # write code here to instantiate layers
        # for example, self.conv = nn.Conv2d(1, 4, 3, 1, 1)
        # creates a conv layer with 1 input channel, 4 output
        # channels, a 3x3 kernel, and stride=padding=1
        self.layer_stack = nn.Sequential()
        inp = 1

        for stage in range(3): ## ANS: Loop over stages
          inp = 1 if stage == 0 else int(math.pow(2,stage+1))
          ## ANS: set input size as power of 2 except as 1 for first stage
          ## i.e. 1,4,8
          out = int(math.pow(2,stage+2))
          ## ANS: set output size as power of 2
          ## i.e. 4,8,16
          for i in range(k//3):
            self.layer_stack.add_module(f"stage{stage+1}conv{i+1}", nn.Conv2d(inp,out,3,1,1))
            self.layer_stack.add_module(f"stage{stage+1}bn{i+1}", nn.BatchNorm2d(out))
            self.layer_stack.add_module(f"stage{stage+1}elu{i+1}", nn.ELU())
            inp = out ## ANS: to move from first layer first stage being 1 -> 4 to second layer first step being 4 -> 4
          self.layer_stack.add_module(f"stage{stage+1}avgpool", nn.AvgPool2d(2,2))

        self.layer_stack.add_module("flatten", nn.Flatten()) ## ANS: flatten before fully connected layer
        self.layer_stack.add_module("fullyconnected", nn.Linear(144,10))

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # insert code to compute sigma
                sigma = 1/(math.sqrt(m.out_channels)*3) ## choosing \sigma = sqrt(1/(C_in*(k^2)))
                m.weight.data.normal_(0, sigma)
                m.bias.data.zero_()

    def forward(self, input):

        # write code here to define how the output u is computed
        # from the input and the model's layers
        # for example, u = self.conv(input) defines u
        # to be simply the output of self.conv given 'input'
        u = self.layer_stack(input)
        return u

Repeat the procedure of finding the maximum number of layers such that the network is still trainable, this time with CNNeluBN.

In [ ]:
k_list = [162, 165]
lr_list = [0.005]

cnnelubn_k_train_test_errs = np.empty((len(k_list), 3))

for lr in lr_list:
  print("Learning rate", lr)
  for idx, k in enumerate(k_list):
    print("\nTraining ELU CNN + BN with {} layers".format(k))
    model = CNNeluBN(k).cuda()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    train_errs, test_errs = train(3, model, criterion, optimizer, train_loader, test_loader)

Learning rate 0.005

Training ELU CNN + BN with 162 layers
Epoch 001/003, Train Error 51.17% || Test Error 31.68%
Epoch 002/003, Train Error 26.97% || Test Error 18.46%
Epoch 003/003, Train Error 14.45% || Test Error 11.16%

Training ELU CNN + BN with 165 layers
Epoch 001/003, Train Error 81.70% || Test Error 66.53%
Epoch 002/003, Train Error 58.88% || Test Error 47.62%
Epoch 003/003, Train Error 36.60% || Test Error 30.09%


### Residual Networks

Finally, you experiment adding residual connections to a CNN.

To implement the model below, you should add a 'skip connection' to 'Conv->BatchNorm->ELU' blocks whenever the shape of the block's input and output are the same: this will be the case for every such block except for the first ones in each stage, as they double the number of channels.

More specifically, you should change $u = ELU(BatchNorm(Conv(x)))$ to $u = ELU(BatchNorm(Conv(x))) + x$, where $x$ and $u$ denote the block's input and output, respectively.

You should take your CNNeluBN implementation and add skip-connections as described above.

Note that there are key differences between the resulting model and the actual ResNet proposed by He et al. in 'Deep Residual Learning for Image Recognition', for example the use of ELU activations instead of ReLU and the exact position of skip-connections.

In [94]:
class ResNet(nn.Module):
    def __init__(self, k):
        super(ResNet, self).__init__()

        # write code here to instantiate layers
        # for example, self.conv = nn.Conv2d(1, 4, 3, 1, 1)
        # creates a conv layer with 1 input channel, 4 output
        # channels, a 3x3 kernel, and stride=padding=1
        self.k = k

        self.layer_stack = nn.Sequential()
        inp = 1

        for stage in range(3): ## ANS: Loop over stages
          inp = 1 if stage == 0 else int(math.pow(2,stage+1))
          ## ANS: set input size as power of 2 except as 1 for first stage
          ## i.e. 1,4,8
          out = int(math.pow(2,stage+2))
          ## ANS: set output size as power of 2
          ## i.e. 4,8,16
          for i in range(k//3):
            self.layer_stack.add_module(f"stage{stage+1}conv{i+1}", nn.Conv2d(inp,out,3,1,1))
            self.layer_stack.add_module(f"stage{stage+1}bn{i+1}", nn.BatchNorm2d(out))
            self.layer_stack.add_module(f"stage{stage+1}elu{i+1}", nn.ELU())
            inp = out ## ANS: to move from first layer first stage being 1 -> 4 to second layer first step being 4 -> 4
          self.layer_stack.add_module(f"stage{stage+1}avgpool", nn.AvgPool2d(2,2))

        self.layer_stack.add_module("flatten", nn.Flatten()) ## ANS: flatten before fully connected layer
        self.layer_stack.add_module("fullyconnected", nn.Linear(144,10))


        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # insert code to compute sigma
                sigma = 1/(math.sqrt(m.out_channels)*3) ## choosing \sigma = sqrt(1/(C_in*(k^2)))
                m.weight.data.normal_(0, sigma)
                m.bias.data.zero_()

    def forward(self, input):

        # write code here to define how the output u is computed
        # from the input and the model's layers
        # for example, u = self.conv(input) defines u
        # to be simply the output of self.conv given 'input'

        out = input

        for i in range(self.k//3):
          l = i*3
          # print(l, self.layer_stack[l])
          # print(0, (k-3))
          # if((l!=0) and (l!=(k-3))):
          if((l!=0)):
            # print("add!")
            out = self.layer_stack[l+2](self.layer_stack[l+1](self.layer_stack[l](out))) + out
          else:
            out = self.layer_stack[l+2](self.layer_stack[l+1](self.layer_stack[l](out)))

        out = self.layer_stack[k](out)

        for i in range((self.k//3), ((self.k*2)//3)):
          l = (i*3)+1
          # print(l, self.layer_stack[l])
          # print((k+1), ((k*2)-2))
          # if(l!=(k+1) and (l!=((k*2)-2))):
          if(l!=(k+1)):
            # print("add!")
            out = self.layer_stack[l+2](self.layer_stack[l+1](self.layer_stack[l](out))) + out
          else:
            out = self.layer_stack[l+2](self.layer_stack[l+1](self.layer_stack[l](out)))

        out = self.layer_stack[(k*2)+1](out)

        for i in range((self.k//3)*2, ((self.k*3)//3)):
          l = (i*3)+2
          # print(l, self.layer_stack[l])
          # print(((k*2)+2), ((k*3)-1))
          # if(l!=((k*2)+2) and (l!=((k*3)-1))):
          if(l!=((k*2)+2)):
            # print("add!")
            out = self.layer_stack[l+2](self.layer_stack[l+1](self.layer_stack[l](out))) + out
          else:
            out = self.layer_stack[l+2](self.layer_stack[l+1](self.layer_stack[l](out)))

        out = self.layer_stack[(k*3)+2](out)
        out = self.layer_stack[(k*3)+3](out)
        out = self.layer_stack[(k*3)+4](out)

        return out

Repeat the procedure of finding the maximum number of layers such that the network is still trainable, this time with the 'ResNet' model.

In [120]:
k_list = [900]
lr_list = [0.005]

resnet_k_train_test_errs = np.empty((len(k_list), 3))

for lr in lr_list:
  print("Learning rate", lr)
  for idx, k in enumerate(k_list):
    print("\nTraining ResNet with {} layers".format(k))
    model = ResNet(k).cuda()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    train_errs, test_errs = train(3, model, criterion, optimizer, train_loader, test_loader)

Learning rate 0.005

Training ResNet with 900 layers
Epoch 001/003, Train Error 63.51% || Test Error 33.50%
Epoch 002/003, Train Error 27.29% || Test Error 21.52%
Epoch 003/003, Train Error 19.16% || Test Error 16.14%


**<font color='blue'>
    Summarize your results and observations regarding the experiments above. What was the maximum number of layers for each of the five models such that training remained successful? Briefly discuss why you think each modification helped/harmed the trainability of deep models.
</font>**

**<font color='red'> --------------------------------------------------------------------- ANSWER (BEGIN) ---------------------------------------------------------------------
</font>**

This was an interesting and insightful experiment to figure out what works and does not work in practice. As someone who had not run a lot of ML models before this, this was a great place to get some insight into differing techniques and how they interact with one another.

We started off with the Tanh model. This was quite similar to the previous assignment and we hit similar depths, where the model worked fine for k=6, but failed at k=9.

The second attempt we made was using the better initialisation as mentioned. This really helped avoid the early vanishing/exploding gradient problems of the Tanh model, and we were able to train close to 40 layers! k=39 seemed to work, while k=42 did not.

Replacing the tanh with ELU activation functions worked even better with us being able to effectively double the number of layers of possible. We are able to train models effectively for k=84, with k=87 failing to converge. This seems to be because compared to tanh, ELU helps a lot. It does not saturate on one end like Tanh does. Also ELU being assymetric around 0 might help a little?

Moving to the next function, we try batch normalisation. This ends up also massively improving our performance, we end up being able to train upto k=162 layers, with k=165 layers failing. Batch normalisation works really really well! This seems to be because again, it stops the exploding gradients, and also takes some of the effect of the sensitivity to the batch size away. It scales all layers to being in a range, and that massively helps stability.

Finally we get to residual networks! This again helps our stability massively as we are always keeping the next conv input to the same order of magnitude as the previous one. The residual also does not allow the signal to ever die down, always keeping some of the past signal alive. This provides us huge leaps in trainability with k=900 being trainable! It takes about 20 minutes to run a single k value at this depth on Colab so I was unable to search for the edge where we fail to converge. Needless to say, resnet is an absolute monster and a winner in terms of depth!

**<font color='red'> ---------------------------------------------------------------------- ANSWER (END) ----------------------------------------------------------------------
</font>**

### Interactions: Batch Norm and Initialization

Intuitively, batch norm should make the model more robust to changes in the magnitude of the network's weights: informally, scaling up all the elements of a conv layer's filters by a factor of 10 would not affect the network's output as long as there is a batch norm layer following such convolution, as the normalization would undo the scaling.

To check how this intuition translates to practical settings, you should change the original 'CNNtanh' model so that it incorporates batch norm layers (like you have done when modifying 'CNNelu' into 'CNNeluBN').

The model below should adopt the naive initialization procedure of sampling from a Gaussian with a deviation of 0.05, not the more sophisticated one that you implemented previously

In [15]:
class CNNtanhBN_oldinit(nn.Module):
    def __init__(self, k):
        super(CNNtanhBN_oldinit, self).__init__()

        # write code here to instantiate layers
        # for example, self.conv = nn.Conv2d(1, 4, 3, 1, 1)
        # creates a conv layer with 1 input channel, 4 output
        # channels, a 3x3 kernel, and stride=padding=1
        self.layer_stack = nn.Sequential()
        inp = 1

        for stage in range(3): ## ANS: Loop over stages
          inp = 1 if stage == 0 else int(math.pow(2,stage+1))
          ## ANS: set input size as power of 2 except as 1 for first stage
          ## i.e. 1,4,8
          out = int(math.pow(2,stage+2))
          ## ANS: set output size as power of 2
          ## i.e. 4,8,16
          for i in range(k//3):
            self.layer_stack.add_module(f"stage{stage+1}conv{i+1}", nn.Conv2d(inp,out,3,1,1))
            self.layer_stack.add_module(f"stage{stage+1}bn{i+1}", nn.BatchNorm2d(out))
            self.layer_stack.add_module(f"stage{stage+1}tanh{i+1}", nn.Tanh())
            inp = out ## ANS: to move from first layer first stage being 1 -> 4 to second layer first step being 4 -> 4
          self.layer_stack.add_module(f"stage{stage+1}avgpool", nn.AvgPool2d(2,2))

        self.layer_stack.add_module("flatten", nn.Flatten()) ## ANS: flatten before fully connected layer
        self.layer_stack.add_module("fullyconnected", nn.Linear(144,10))

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data.normal_(0, 0.05)
                m.bias.data.zero_()

    def forward(self, input):

        # write code here to define how the output u is computed
        # from the input and the model's layers
        # for example, u = self.conv(input) defines u
        # to be simply the output of self.conv given 'input'
        u = self.layer_stack(input)

        return u

Repeat the procedure of finding the maximum number of layers such that the network is still trainable, this time with CNNeluBN_oldinit.

In [21]:
k_list = [45, 48, 51]
lr_list = [0.05]

cnntanhinit_k_train_test_errs = np.empty((len(k_list), 3))

for lr in lr_list:
  print("Learning rate", lr)
  for idx, k in enumerate(k_list):
    print("\nTraining Tanh CNN + BN + naive init with {} layers".format(k))
    model = CNNtanhBN_oldinit(k).cuda()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    train_errs, test_errs = train(3, model, criterion, optimizer, train_loader, test_loader)

Learning rate 0.05

Training Tanh CNN + BN + naive init with 45 layers
Epoch 001/003, Train Error 27.61% || Test Error 8.57%
Epoch 002/003, Train Error 7.40% || Test Error 5.10%
Epoch 003/003, Train Error 4.43% || Test Error 3.17%

Training Tanh CNN + BN + naive init with 48 layers
Epoch 001/003, Train Error 21.44% || Test Error 9.07%
Epoch 002/003, Train Error 6.47% || Test Error 3.58%
Epoch 003/003, Train Error 3.65% || Test Error 3.15%

Training Tanh CNN + BN + naive init with 51 layers
Epoch 001/003, Train Error 19.20% || Test Error 7.35%
Epoch 002/003, Train Error 86.16% || Test Error 88.65%
Epoch 003/003, Train Error 88.88% || Test Error 88.65%


**<font color='blue'>
    Compare CNNtanh (model with naive initialization and no batch norm), CNNtanh_newinit (model with better initialization and no batch norm), and CNNtanhBN_oldinit (model with naive initialization and batch norm), in terms of how deep each could be while being trainable, and discuss your thoughts one how batch norm interacts with the way parameters are initialized.
</font>**

**<font color='red'> --------------------------------------------------------------------- ANSWER (BEGIN) ---------------------------------------------------------------------
</font>**

CNNtanh is by far the worst one in terms of being trainable, only ever reaching 6 layers. CNNtanh_newinit is much better by being able to go up to k=39. Compared to these two CNNtanhBN_oldinit is much closer to the second one. Informally, the intuition of BN being able to "reverse" the better inititalisation seems to hold true, and BN also seems to bring its own improvements along with that. It is interesting how taking a step "backward" to worse initialisation and adding BatchNorm, moves our performance not only back to the better initialisation method but actually goes slightly further than that and gives us k = 48 being trainable!

**<font color='red'> ---------------------------------------------------------------------- ANSWER (END) ----------------------------------------------------------------------
</font>**

### Interactions: Batch Norm and Residual Connections

Lastly, implement and train a CNN with residual connections but without batch normalization layers -- the goal here is to check how residuals interact with normalization.

The model below should be exactly like ResNet, except that it should not have batch norm layers.

In [118]:
class ResNet_noBN(nn.Module):
    def __init__(self, k):
        super(ResNet_noBN, self).__init__()

        # write code here to instantiate layers
        # for example, self.conv = nn.Conv2d(1, 4, 3, 1, 1)
        # creates a conv layer with 1 input channel, 4 output
        # channels, a 3x3 kernel, and stride=padding=1
        self.k = k

        self.layer_stack = nn.Sequential()
        inp = 1

        for stage in range(3): ## ANS: Loop over stages
          inp = 1 if stage == 0 else int(math.pow(2,stage+1))
          ## ANS: set input size as power of 2 except as 1 for first stage
          ## i.e. 1,4,8
          out = int(math.pow(2,stage+2))
          ## ANS: set output size as power of 2
          ## i.e. 4,8,16
          for i in range(k//3):
            self.layer_stack.add_module(f"stage{stage+1}conv{i+1}", nn.Conv2d(inp,out,3,1,1))
            # self.layer_stack.add_module(f"stage{stage+1}bn{i+1}", nn.BatchNorm2d(out))
            self.layer_stack.add_module(f"stage{stage+1}elu{i+1}", nn.ELU())
            inp = out ## ANS: to move from first layer first stage being 1 -> 4 to second layer first step being 4 -> 4
          self.layer_stack.add_module(f"stage{stage+1}avgpool", nn.AvgPool2d(2,2))

        self.layer_stack.add_module("flatten", nn.Flatten()) ## ANS: flatten before fully connected layer
        self.layer_stack.add_module("fullyconnected", nn.Linear(144,10))

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # insert code to compute sigma
                sigma = 1/(math.sqrt(m.out_channels)*3) ## choosing \sigma = sqrt(1/(C_in*(k^2)))
                m.weight.data.normal_(0, sigma)
                m.bias.data.zero_()

    def forward(self, input):

        # write code here to define how the output u is computed
        # from the input and the model's layers
        # for example, u = self.conv(input) defines u
        # to be simply the output of self.conv given 'input'

        out = input

        for i in range(self.k//3):
          l = i*2
          # print(l, self.layer_stack[l])
          # print(0, (k-3))
          # if((l!=0) and (l!=(k-3))):
          if((l!=0)):
            # print("add!")
            out = self.layer_stack[l+1](self.layer_stack[l](out)) + out
          else:
            out = self.layer_stack[l+1](self.layer_stack[l](out))

        out = self.layer_stack[int((2*k)/3)](out)

        for i in range((self.k//3), ((self.k*2)//3)):
          l = (i*2)+1
          # print(l, self.layer_stack[l])
          # print((k+1), ((k*2)-2))
          # if(l!=(k+1) and (l!=((k*2)-2))):
          if(l!=(int((2*k)/3)+1)):
            # print("add!")
            out = self.layer_stack[l+1](self.layer_stack[l](out)) + out
          else:
            out = self.layer_stack[l+1](self.layer_stack[l](out))

        out = self.layer_stack[int((k*4)/3)+1](out)

        for i in range((self.k//3)*2, ((self.k*3)//3)):
          l = (i*2)+2
          # print(l, self.layer_stack[l])
          # print(((k*2)+2), ((k*3)-1))
          # if(l!=((k*2)+2) and (l!=((k*3)-1))):
          if(l!=(int((k*4)/3)+2)):
            # print("add!")
            out = self.layer_stack[l+1](self.layer_stack[l](out)) + out
          else:
            out = self.layer_stack[l+1](self.layer_stack[l](out))

        out = self.layer_stack[int(k*2)+2](out)
        out = self.layer_stack[int(k*2)+3](out)
        out = self.layer_stack[int(k*2)+4](out)

        return out

Repeat the procedure of finding the maximum number of layers such that the network is still trainable, this time with ResNet_noBN.

In [129]:
k_list = [18, 21, 24]
lr_list = [0.005]

resnet_k_train_test_errs = np.empty((len(k_list), 3))

for lr in lr_list:
  print("Learning rate", lr)
  for idx, k in enumerate(k_list):
    print("\nTraining ResNet w/o BN with {} layers".format(k))
    model = ResNet_noBN(k).cuda()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    train_errs, test_errs = train(3, model, criterion, optimizer, train_loader, test_loader)

Learning rate 0.005

Training ResNet w/o BN with 18 layers
Epoch 001/003, Train Error 12.82% || Test Error 2.49%
Epoch 002/003, Train Error 2.65% || Test Error 2.27%
Epoch 003/003, Train Error 1.95% || Test Error 1.65%

Training ResNet w/o BN with 21 layers
Epoch 001/003, Train Error 90.08% || Test Error 90.20%
Epoch 002/003, Train Error 90.13% || Test Error 90.20%
Epoch 003/003, Train Error 90.13% || Test Error 90.20%

Training ResNet w/o BN with 24 layers
Epoch 001/003, Train Error 10.99% || Test Error 2.54%
Epoch 002/003, Train Error 2.28% || Test Error 1.91%
Epoch 003/003, Train Error 1.73% || Test Error 1.56%


**<font color='blue'>
    Compare ResNet and ResNet_noBN in terms of how deep each could be while being trainable, and discuss your thoughts one how batch norm interacts with residual connections.
</font>**

**<font color='red'> --------------------------------------------------------------------- ANSWER (BEGIN) ---------------------------------------------------------------------
</font>**

resnet without the batchnorm layer really starts to falter quickly! My guess is that batch norm is super helpful with residual connections by forcing things back to being in the same-ish range of values. Without batch normalisation, adding residuals just blows things up more quickly, and the deeper the network, the worse it gets!

**<font color='red'> ---------------------------------------------------------------------- ANSWER (END) ----------------------------------------------------------------------
</font>**

### (Optional) Multiple Loss Heads

In this optional section, your goal is to incorporate the idea of having multiple loss heads throughout the network, distributed across its depth.

For the CNNelu_multihead model below, you should take the CNNelu model that you implemented previously and add two additional classification heads, connected to the outputs of stages 1 and 2.

More specifically, the outputs of stages 1 and 2, with shapes 4x14x14 and 8x7x7, should be connected to new fully-connected layers that map them to a 10-dimensional vector (logits for the 10 MNIST classes). The network should output three logit vectors (the original one at the end of the network plus the two new ones) instead of just one, and the loss should be computed as the average of the cross entropies between the true target and each of the three predictions.

Note that you will likely have to change the implementation of train_epoch() and test() to accomodate the fact that this model will output three logit vectors instead of one.

In [ ]:
class CNNelu_multihead(nn.Module):
    def __init__(self, k):
        super(CNNelu_multihead, self).__init__()

        # write code here to instantiate layers
        # for example, self.conv = nn.Conv2d(1, 4, 3, 1, 1)
        # creates a conv layer with 1 input channel, 4 output
        # channels, a 3x3 kernel, and stride=padding=1


        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # insert code to compute sigma
                sigma =
                m.weight.data.normal_(0, sigma)
                m.bias.data.zero_()

    def forward(self, input):
        # write code here to define how the output u is computed
        # from the input and the model's layers
        # for example, u = self.conv(input) defines u
        # to be simply the output of self.conv given 'input'

        return u1, u2, u3

Repeat the procedure of finding the maximum number of layers such that the network is still trainable, this time with CNNelu_multihead.

In [ ]:
k =
lr =

print("\nTraining ELU CNN + multiloss with {} layers".format(k))
model = CNNelu_multihead(k).cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
train_errs, test_errs = train(3, model, criterion, optimizer, train_loader, test_loader)

**<font color='blue'>
    Did the adoption of multiple loss heads help train deeper models? How did it compare to the adoption of batch normalization, in terms of how deeper each of the two approaches enabled the network to be while staying trainable?
</font>**

**<font color='red'> --------------------------------------------------------------------- ANSWER (BEGIN) ---------------------------------------------------------------------
</font>**

**<font color='red'> ---------------------------------------------------------------------- ANSWER (END) ----------------------------------------------------------------------
</font>**